In [ ]:
# 필요한 라이브러리를 설치합니다.
!pip install datasets==3.2.0 # 데이터셋 로드 및 처리 라이브러리
!pip install trl==0.13.0  # Transformer 모델을 위한 강화 학습 라이브러리
!pip install peft==1.26.4  # 파라미터 효율적인 미세 조정을 위한 라이브러리
!pip install -U bitsandbytes==0.45.0  # 양자화 및 메모리 효율성을 위한 라이브러리

In [ ]:
# 필요한 모듈을 임포트합니다.
from datasets import load_dataset, Dataset
from pprint import pprint
import transformers
from peft import prepare_model_for_kbit_training
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm
import json
import pickle
from peft import LoraConfig, get_peft_model
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# pickle 모듈을 사용하여 청킹한 “전자금융거래.pk” 바이너리 파일 데이터를 불러옵니다.
data_list = pickle.load(open('전자금융거래.pk', 'rb'))

In [ ]:
# 사용할 사전 학습된 모델의 ID를 지정합니다.
model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

# BitsAndBytesConfig를 사용하여 4-bit 양자화 설정을 구성합니다.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 토크나이저와 모델을 불러옵니다.
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"": 0})

# 모델의 기울기 체크포인팅을 활성화하여 메모리 사용을 줄입니다.
model.gradient_checkpointing_enable()

# 모델을 K-bit 양자화 훈련에 적합하도록 준비합니다.
model = prepare_model_for_kbit_training(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# 불러온 데이터 리스트를 기반으로 데이터셋 객체를 생성하고 텍스트 데이터를 토큰화 합니다.
data = Dataset.from_list([{"text": i} for i in data_list], split="train")
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 75
})

In [ ]:
# LoraConfig를 사용하여 LoRA 설정을 구성합니다.
config = LoraConfig(
    r=8,
    lora_alpha=32,  # LoRA 스케일링
    lora_dropout=0.05,  # 드롭아웃 비율
    bias="none",
    task_type="CAUSAL_LM"  # 생성 모델 학습 유형으로 지정
)

# 설정된 LoRA 구성을 사용하여 모델을 PEFT 모델로 변환합니다.
model = get_peft_model(model, config)

In [ ]:
# 학습 전 모델 테스트를 위해 입력 텍스트를 생성해 결과를 확인합니다.
input_text = "전자지급수단이 뭐야?"
gened = model.generate(
      **tokenizer(input_text, return_tensors='pt', return_token_type_ids=False).to(model.device),
      max_new_tokens=128,
      early_stopping=True,
      do_sample=True,
      eos_token_id=2,
)
print(tokenizer.decode(gened[0]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [ ]:
print(tokenizer.decode(gened[0]))

<|begin_of_text|>전자지급수단이 뭐야?」라고 묻는 것과, 「전자지급수는 어디에 있나요?」라는 질문이 서로 다른 맥락으로 파악된다. 첫 질문은 전자지급수가 전자기기에서 어떻게 작동하는지를 이해하고 싶다는 의미로, 두 번째 질문은 전자지급수가 어디에 있는지를 물어하는 것과 같다. 따라서, 전자지급수에 대한 이해를 높이기 위해서는 두 질문 모두에 답해야 한다. 전자지급수란 무엇일까? 전자지급수는 전자기기의 입력단과 출력단 사이에서 전기 신호를 변환하는 역할을 하는 요소다. 이는 전자기기에서 전기 신호를 받아들이지 않은 장치나 기기에서 전기 신호를 보내는 장치로 변환하는 역할을 한다. 예를 들어, 전자기기에서 전기 신호를 받아 컴퓨터로 전송하는 과정에서 전자지급수는 중요한 역할을 한다. 전자지급수는 전자기기의 기본 구성 요소 중 하나로, 전자기기의 작동을 가능하게 하는 중요한 요소다. 전자지급수가 작동하지 않으면


In [ ]:
# 패딩 토큰을 엔드 오브 시퀀스(end-of-sequence) 토큰으로 설정
tokenizer.pad_token = tokenizer.eos_token

# 데이터셋의 총 행(row) 수를 계산하여 data_len 변수에 저장
data_len = data.num_rows

# Trainer 클래스를 사용하여 모델 학습을 설정
trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        warmup_steps=200,
        num_train_epochs=3,
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="qlora_output",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

# 학습 중 메모리 사용량을 최적화하기 위해 캐시 기능을 비활성화
model.config.use_cache = False
trainer.train()  # 설정된 인자와 함께 학습을 시작


model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.155600
20,2.136000
30,2.083700
40,1.986300
50,1.940500
60,1.823600
70,1.770600
80,1.590800
90,1.459500
100,1.408300


In [ ]:
# 학습 후 입력 텍스트로 다시 텍스트 생성을 수행하여 결과를 확인
input_text = "전자지급수단이 뭐야?"
gened = model.generate(
      **tokenizer(input_text, return_tensors='pt', return_token_type_ids=False).to(model.device),
      max_new_tokens=256,
      early_stopping=True,
      do_sample=True,
      eos_token_id=128001,
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(tokenizer.decode(gened[0]))

<|begin_of_text|>전자지급수단이 뭐야? 
- 전자지급수단은 상품이나 서비스를 구입하는 과정에서 사용되는 전자적 또는 정보적 수단을 말한다. 
- 예를 들어, 전자화폐, 전자지급수단(예: 선불카드, 선불전화카드), 전자지급수단(예: 모바일 지불수단) 등이 있다. 
- 이러한 전자지급수단은 현금과 달리 사용자의 신원을 확인하는 과정이 간단하고, 사용자의 개인정보를 보호할 수 있게 하여 
    금융 거래의 안전성과 효율성을 높일 수 있다. 
- 전자지급수단은 다양한 금융 서비스와 결합하여 사용할 수 있으며, 이를 통해 금융 서비스의 다양성과 효율성을 높일 수 있다. 
- 예를 들어, 선불카드와 같은 전자지급수단을 사용하면, 선불금액에 해당하는 금액을 지불할 때마다 현금을 지불하지 않고도 
    이용할 수 있다. 또한, 선불금액이 부족한 경우에는 추가금을 지불할 수 있으며, 선불금액이
